In [ ]:
from datar.all import *
import pandas as pd

In [ ]:
df = pd.read_csv("./ManicTimeData_2023-06-04.csv")
df

In [ ]:
df["Date"] = df.Start.apply(lambda x:x[0:10]) >> as_date()
df['Duration'] = pd.to_timedelta(df['Duration'])
df['Start'] = pd.to_datetime(df['Start'],format='%Y-%m-%dT%H:%M:%S.%f')
df['End'] = pd.to_datetime(df['End'],format='%Y-%m-%dT%H:%M:%S.%f')
df

In [ ]:
def midnight_split(df):
    selected = df >> filter(df.Start.apply(lambda x:x.date()) != df.End.apply(lambda x:x.date()))
    
    for row in selected.itertuples():
        index = row.Index
        name = row.Name
        start1 = row.Start
        start2 = pd.to_datetime(start1.date() + pd.Timedelta(days=1))
        end1 = start2
        end2 = row.End
        duration1 = end1 - start1
        duration2 = end2 - start2
        process = row.Process
        date1 = pd.to_datetime(row.Date)
        date2 = pd.to_datetime(start1.date() + pd.Timedelta(days=1))
        df.loc[index] = [name,start1,end1,duration1,process,date1]
        df.loc[index+0.5] = [name,start2,end2,duration2,process,date2]
    df = df.sort_index().reset_index(drop=True)
    return df
df = midnight_split(df)
df.loc[750:760]

In [ ]:
df['Week'] = df.Date.apply(lambda x:f'{x.year}-{x.week}')
df['Month'] = df.Date.apply(lambda x:f'{x.year}-{x.month}')
df['Year'] = df.Date.apply(lambda x:f'{x.year}')
df

In [ ]:
df.to_parquet('data.parquet',engine="pyarrow")

In [ ]:
import plotly.express as px

In [ ]:
from pipda import register_verb,Context

@register_verb(context=Context.EVAL)
def vmap(data,func):
    return map(func,data)

@register_verb(context=Context.EVAL)
def vjoin(data,string):
    return string.join(data)

#TODO:格式检查
@register_verb(cls = str)
def duration_to_seconds(time:str):
    hour,minute,second = time.split(":") >> vmap(int)
    return hour * 3600 + minute * 60 + second

@register_verb(cls = int)
def seconds_to_duration(time:int):
    hour = time // 3600
    minute = (time - hour * 3600) // 60
    seconds = time - hour * 3600 - minute * 60
    return f'{hour:0>2}:{minute:0>2}:{seconds:0>2}'

@register_verb(cls=pd.Timedelta)
def timedelta_to_hours(time:pd.Timedelta):
    days = time.days
    seconds = time.seconds
    hours = days * 24 + round(seconds / 3600,2)
    return hours

"01:48:06" >> duration_to_seconds() >> seconds_to_duration()

In [ ]:

#每日活跃时长
def active_duration_per_day():
    df_active = df >> group_by(f.Date) >> summarise(Active = sum(f.Duration)) 
    df_active['Active'] = df_active.Active.apply(timedelta_to_hours)
    return df_active
active_df = active_duration_per_day()
active_df

In [ ]:
fig = px.bar(active_df, x='Date', y='Active')
fig.show()

In [ ]:
#TODO:合并成一个类

#TODO:优化星期表示
# 每星期活跃时长
def active_duration_per_week():
    df_active = df >> group_by(f.Week) >> summarise(active = sum(f.Duration)) 
    return df_active
# 每月活跃时长
def active_duration_per_month():
    df_active = df >> group_by(f.Month) >> summarise(active = sum(f.Duration)) 
    return df_active
# 每年活跃市场
def active_duration_per_year():
    df_active = df >> group_by(f.Year) >> summarise(active = sum(f.Duration)) 
    return df_active
def active_duration_from_to(date_from,date_to):
    df_active = df >> filter(f.Date >= pd.to_datetime(date_from) and f.Date <= pd.to_datetime(date_to)) >> summarise(active = sum(f.Duration)) 
    return df_active
def active_duration_per_week_from_to(date_from,date_to):
    df_active = df >> filter(f.Date >= pd.to_datetime(date_from) and f.Date <= pd.to_datetime(date_to)) >> group_by(f.Week) >> summarise(active = sum(f.Duration)) 
    return df_active

In [ ]:
active_duration_from_to('2023-1-5','2023-2-8')

In [ ]:
active_duration_per_week_from_to('2023-1-5','2023-2-8')

In [ ]:
active_duration_per_month(),active_duration_per_week(),active_duration_per_year()

In [ ]:
def apps_per_day():
    pass